In [5]:
# cost hamiltonian
import numpy as np

# Define the cost Hamiltonian (example: H_C = Z_0 + Z_1 + Z_0 Z_1)
Z = np.array([[1, 0], [0, -1]])  # Pauli-Z matrix
I = np.eye(2)  # Identity matrix

# Construct H_C for a 2-qubit system
H_C = np.kron(Z, I) + np.kron(I, Z) + np.kron(Z, Z)

# Diagonalize the Hamiltonian
eigenvalues, eigenvectors = np.linalg.eig(H_C)

# Eigenenergies are the eigenvalues of H_C
eigenenergies = np.real(eigenvalues)  # Take the real part (imaginary part should be zero)
print("Eigenenergies:", eigenenergies)

Eigenenergies: [ 3. -1. -1. -1.]


In [8]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2

ImportError: cannot import name 'SamplerV2' from 'qiskit_aer.primitives' (/pscratch/sd/g/gzquse/QML_2025/.conda/lib/python3.10/site-packages/qiskit_aer/primitives/__init__.py)

In [ ]:
sim = AerSimulator()
# --------------------------
# Simulating using estimator
#---------------------------
from qiskit_aer.primitives import EstimatorV2

psi1 = transpile(RealAmplitudes(num_qubits=2, reps=2), sim, optimization_level=0)
psi2 = transpile(RealAmplitudes(num_qubits=2, reps=3), sim, optimization_level=0)

H1 = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
H2 = SparsePauliOp.from_list([("IZ", 1)])
H3 = SparsePauliOp.from_list([("ZI", 1), ("ZZ", 1)])

theta1 = [0, 1, 1, 2, 3, 5]
theta2 = [0, 1, 1, 2, 3, 5, 8, 13]
theta3 = [1, 2, 3, 4, 5, 6]

estimator = EstimatorV2()

# calculate [ [<psi1(theta1)|H1|psi1(theta1)>,
#              <psi1(theta3)|H3|psi1(theta3)>],
#             [<psi2(theta2)|H2|psi2(theta2)>] ]
job = estimator.run(
    [
        (psi1, [H1, H3], [theta1, theta3]),
        (psi2, H2, theta2)
    ],
    precision=0.01
)
result = job.result()
print(f"expectation values : psi1 = {result[0].data.evs}, psi2 = {result[1].data.evs}")

In [ ]:


# Number of qubits (photon transmissions)
num_bits = 100

# Step 1: Alice prepares qubits
alice_bits = np.random.randint(2, size=num_bits)  # Random bits 0 or 1
alice_bases = np.random.randint(2, size=num_bits)  # 0 = Z-basis, 1 = X-basis

# Step 2: Create Qiskit circuits for each qubit
circuits = []
for bit, basis in zip(alice_bits, alice_bases):
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)  # Apply X gate for bit=1
    if basis == 1:
        qc.h(0)  # Apply H if Alice's basis is X
    circuits.append(qc)

# Step 3: Bob randomly chooses bases and measures
bob_bases = np.random.randint(2, size=num_bits)

# Step 4: Execute quantum circuits
backend = Aer.get_backend('qasm_simulator')
measurements = []
for i in range(num_bits):
    qc = circuits[i]
    if bob_bases[i] == 1:
        qc.h(0)  # Bob applies H if measuring in X-basis
    qc.measure(0, 0)
    result = execute(qc, backend, shots=1).result()
    counts = result.get_counts()
    measured_bit = int(list(counts.keys())[0])  # Get measured bit
    measurements.append(measured_bit)

# Step 5: Alice and Bob discard mismatched basis measurements
valid_indices = np.where(alice_bases == bob_bases)[0]
key_alice = alice_bits[valid_indices]
key_bob = np.array(measurements)[valid_indices]

# Step 6: Display the shared key
print("Alice's raw key:", key_alice)
print("Bob's raw key:  ", key_bob)

# Final key after removing errors (if any)
final_key = key_alice[key_alice == key_bob]  # Remove errors
print("Final shared key:", final_key)
